In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Download NLTK resources (do it once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
file=r"C:\Users\jayen\Text-sentiment-analysis-general-purpose\artifacts\emotions.csv"
n=100000
df=pd.read_csv(file)
df

,text,label
0,i feel awful about it too because it s my job ...,0
1,im alone i feel awful,0
2,ive probably mentioned this before but i reall...,1
3,i was feeling a little low few days back,0
4,i beleive that i am much more sensitive to oth...,2
...,...,...
99995,i continued to feel discouraged and downtrodde...,0
99996,i feel like i am afraid of not being in contro...,4
99997,i feel hated and ignored now,3
99998,i feel sort of dazed and unfocused and really ...,5


In [ ]:
df.to_csv('emotions.csv',index=False)

In [ ]:
lables
0=sadness
1=joy
2=love
3=anger
4=Fear 
5=surprise

In [5]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [3]:
df['label'].value_counts()

label
1    33675
0    29121
3    13722
4    11560
2     8292
5     3630
Name: count, dtype: int64

using ml models

In [ ]:
#NOW ILL DO SOME PREPROCESSING STEPS FOR THAT ILL CREATE A FUNCTION
def preprocessing(q):

  q=str(q).lower().strip()#strip gives a string in return if u use split it will give a list of words
  #The strip() method in Python is used to remove leading and trailing characters (whitespace characters by default) from a string before " Hello, world! " after"Hello, world!"
   # Replace certain special characters with their string equivalents
  q = q.replace('%', 'percent')
  q = q.replace('$', 'dollar ')
  q = q.replace('₹', 'rupee ')
  q = q.replace('€', 'euro ')
  q = q.replace('@', 'at')
  q=q.replace('&','and')

  q = re.sub(r'([0-9]+)000000000', r'\1b', re.sub(r'([0-9]+)000000', r'\1m', re.sub(r'([0-9]+)000', r'\1k', q)))#Input: "1500000000 Output: "1.5b" it will do this
  q=re.sub(r'\b(?:\d{1,4}[/-]\d{1,2}[/-]\d{1,4}|\d{1,2}[/-]\d{1,2}[/-]\d{2,4})\b','',q)#for removing dates
  q=re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b','',q)#removing email
  q= re.sub(r'https?://\S+|www\.\S+','',q)#removing urls



    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
  contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }
  q_decontracted=[]

  q_decontracted = [contractions[i] if i in contractions else i for i in q.split()]


  #now i join this word into q
  q=' '.join(q_decontracted)
  q = q.replace("'ve", " have")
  q = q.replace("n't", " not")
  q = q.replace("'re", " are")
  q = q.replace("'ll", " will")
  # Removing HTML tags
  q = BeautifulSoup(q)
  q = q.get_text()
  # Remove punctuations like ,.""[]
  q =re.sub(r'[^\w\s{}\/<>\'"\-_=+()*~:;]', ' ', q)

  q = word_tokenize(q)  # Convert to lowercase

  # Remove stopwords and non-alphabetic characters, perform lemmatization
  q = [
        lemmatizer.lemmatize(word) for word in q
        if word not in stop_words
    ]



  return ' '.join(q)#ALWAYS REMEMBER IN ORDER TO USE WORD2VEC UR SENTENSES SHOULD BE IN TOKKENS OR WORDS BUT IN THIS CASE WE HAVE TO TRAIN OUR MODEL WE HAVE TO TOOKENIZE
  #THE WORDS SO WE USE JOIN THEN AFTER WE TOKKENIZE IT WONT BE ABLE TO CONCATINATE LIST OF STRINGS IMPORTANT. contain lists instead of strings.


In [ ]:
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Bidirectional
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['preprocessed'])
sequences = tokenizer.texts_to_sequences(df['preprocessed'])

In [ ]:
max_length = 200  # Define your maximum sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_length)#THIS WILL BE MY ACTUAL DATA THAT'LL TRAIN TEST AND SPLIT

In [ ]:
word_index = tokenizer.word_index#THESE ARE MY UNIQUE WORDS WITH ITS UNIQUE INDEX
num_words = len(word_index)+1

In [ ]:

## Creating model
from tensorflow.keras.layers import Dropout
embedding_vector_features=200 ##features representation each word wil get convert into a vector of 300
model=Sequential()
model.add(Embedding(num_words,embedding_vector_features,input_length=max_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(50,return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(6,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

import numpy as np
X_final=np.array(padded_sequences)
y_final=np.array(df['label'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', mode='max', verbose=1, baseline=0.95)

In [ ]:
model.fit(X_final,y_final, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
predicted_labels = [max(enumerate(pred), key=lambda x: x[1])[0] for pred in y_pred]

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f'Accuracy: {accuracy}')

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

In [ ]:
import os

folder_path = r'C:\Users\jayen\Text-sentiment-analysis-general-purpose\models'

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

model.save(r'C:\Users\jayen\Text-sentiment-analysis-general-purpose\models\my_model')